Malik and Pitt resampling:

We are assuming that we have a set of uniforms that we use
to invert the cdf $\tilde{F}(x)$ above. So, we have the set of N, sorted in ascending order, uniform variates u1 < · · · < uN . The generation of the, possibly stratified, uniforms is discussed later in this appendix.


The pseudo-algorithm given below samples the index corre- sponding to the region which are stored as r1, r2, . . . , rN , and also produces a new set of uniforms, u∗1, . . . , u∗N 

set s = 0, j = 1; for {i = 0 : N}
{
    s = s + πi;
    while(uj ≤s & j≤N){
        rj =i;
        u∗j = (uj − (s − πi))/πi 
        j=j+1
}

For the selected regions where $r^j = 0$, we set $x_j^* = x^{(1)}$ and when $r^j = N$ we set $x_j^* = x^{(N)}$. Otherwise set: 
(x(rj+1) − x(rj)) × u∗j + x(rj
$x_j^* = (x^{(r^j+1)} - x^{(r^j)}) * u_j^* + x^{(r^j)}$

In [1]:
import numpy as np

In [6]:
s = 0
j = 1
N = 10

# normalized weights
w = np.random.uniform(size=N)
w = w/sum(w)

r = np.zeros(N+1)
u_star = np.zeros(N+1)
np.random.uniform(size=N)

In [ ]:
for i in range(N):
    s = s + w[i]
    while (u[j] < s) and (j <= N):
        r[j] = i
        u_star[j] = (u[j] - (s - w[i]))/w[i]
        j += 1